In [1]:
# materials/stack

import pandas as pd
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import zscore

data = pd.read_parquet('data/raw/data_v1.parquet')
forecast = pd.read_parquet('data/processed/data_v1_forecast.parquet')
scores = pd.DataFrame()

In [2]:
forecast

,quarter,quarter_label,ig_maus,ig_revenue,tk_maus,tk_revenue
0,2024-03-31,Q2 2024,2221735781,12735807893,1871336709,4527462823
1,2024-06-30,Q3 2024,2242448858,12929389484,1631989456,3749413402
2,2024-09-30,Q4 2024,2276922459,13251575055,1576689951,3569650540
3,2024-12-31,Q1 2025,2369122780,14113266663,1495697320,3306366674


## Scoring: Last Period

In [3]:
last_results = forecast.tail(1)
ig_last_results = last_results.drop(columns=['tk_maus', 'tk_revenue'])
tk_last_results = last_results.drop(columns=['ig_maus', 'ig_revenue'])

In [4]:
display(ig_last_results, tk_last_results)

,quarter,quarter_label,ig_maus,ig_revenue
3,2024-12-31,Q1 2025,2369122780,14113266663


,quarter,quarter_label,tk_maus,tk_revenue
3,2024-12-31,Q1 2025,1495697320,3306366674


In [5]:
# Formatar os números com separadores de milhar
lp_ig_revenue = f"{ig_last_results['ig_revenue'].values[0]:,}"
lp_tk_revenue = f"{tk_last_results['tk_revenue'].values[0]:,}"
lp_difference = f"{ig_last_results['ig_revenue'].values[0] - tk_last_results['tk_revenue'].values[0]:,}"

print('Instagram revenues forecast:', lp_ig_revenue)
print('TikTok revenues forecast:', lp_tk_revenue)
print('Difference between Instagram and TikTok:', lp_difference)
print()
if ig_last_results['ig_revenue'].values[0] > tk_last_results['tk_revenue'].values[0]:
    print('Instagram pretended to have more revenues.')
else:
    print('TikTok pretended to have more revenues.')

Instagram revenues forecast: 14,113,266,663
TikTok revenues forecast: 3,306,366,674
Difference between Instagram and TikTok: 10,806,899,989

Instagram pretended to have more revenues.


In [6]:
def calculate_app_scores(data, w1=0.4, w2=0.6):
    """
    Calcula os scores de dois aplicativos baseados em receitas e MAUs.

    Parâmetros:
    - data: DataFrame com colunas 'ig_maus', 'ig_revenue', 'tk_maus', 'tk_revenue'.
    - w1: Peso para a receita (valor entre 0 e 1).
    - w2: Peso para MAUs (valor entre 0 e 1).

    Retorna:
    - DataFrame com os scores dos aplicativos.
    """
    # Normalização
    data['ig_normalized_revenue'] = (data['ig_revenue'] - data['ig_revenue'].min()) / (data['ig_revenue'].max() - data['ig_revenue'].min())
    data['ig_normalized_maus'] = (data['ig_maus'] - data['ig_maus'].min()) / (data['ig_maus'].max() - data['ig_maus'].min())
    data['tk_normalized_revenue'] = (data['tk_revenue'] - data['tk_revenue'].min()) / (data['tk_revenue'].max() - data['tk_revenue'].min())
    data['tk_normalized_maus'] = (data['tk_maus'] - data['tk_maus'].min()) / (data['tk_maus'].max() - data['tk_maus'].min())

    # Cálculo do Score
    data['ig_score'] = w1 * data['ig_normalized_revenue'] + w2 * data['ig_normalized_maus']
    data['tk_score'] = w1 * data['tk_normalized_revenue'] + w2 * data['tk_normalized_maus']

    return data[['quarter', 'ig_score', 'tk_score']]

teste = calculate_app_scores(data)

In [7]:
scoring = data.tail(1)

In [8]:
scoring

,quarter,quarter_label,ig_maus,ig_revenue,tk_maus,tk_revenue,ig_normalized_revenue,ig_normalized_maus,tk_normalized_revenue,tk_normalized_maus,ig_score,tk_score
20,2024-01-01,Q1 2024,2231000000,14472000000,1666000000,4583000000,0.92956,1.0,0.916211,1.0,0.971824,0.966485


In [9]:
forecast

,quarter,quarter_label,ig_maus,ig_revenue,tk_maus,tk_revenue
0,2024-03-31,Q2 2024,2221735781,12735807893,1871336709,4527462823
1,2024-06-30,Q3 2024,2242448858,12929389484,1631989456,3749413402
2,2024-09-30,Q4 2024,2276922459,13251575055,1576689951,3569650540
3,2024-12-31,Q1 2025,2369122780,14113266663,1495697320,3306366674


In [10]:
teste

,quarter,ig_score,tk_score
0,2019-01-01,0.000000,0.000000
1,2019-04-01,0.048961,0.022334
2,2019-07-01,0.080105,0.049817
3,2019-10-01,0.136805,0.087756
4,2020-01-01,0.155287,0.136086
5,2020-04-01,0.208193,0.212307
6,2020-07-01,0.255944,0.208787
7,2020-10-01,0.347915,0.257348
8,2021-01-01,0.381810,0.278182
9,2021-04-01,0.447312,0.341765


In [11]:
forecast

,quarter,quarter_label,ig_maus,ig_revenue,tk_maus,tk_revenue
0,2024-03-31,Q2 2024,2221735781,12735807893,1871336709,4527462823
1,2024-06-30,Q3 2024,2242448858,12929389484,1631989456,3749413402
2,2024-09-30,Q4 2024,2276922459,13251575055,1576689951,3569650540
3,2024-12-31,Q1 2025,2369122780,14113266663,1495697320,3306366674


In [12]:
def calculate_scores(df, weight_revenue, weight_maus, x: str, y: str): 
    try:
        # Validar pesos
        # if weight_revenue + weight_maus != 1:
        #    raise ValueError("Os pesos devem somar 1.")

        # Normalizar utilizando Z-score
        df[f'{x}_z'] = zscore(df[f'{x}'])
        df[f'{y}_z'] = zscore(df[f'{y}'])
        

        # Calcular o score composto
        df['score'] = (df[f'{y}_z'] * weight_revenue) + (df[f'{x}_z'] * weight_maus)

        return df
    except Exception as e:
        print(f"Erro ao calcular scores: {e}")

calculate_scores(forecast, 0.5, 0.4, 'ig_maus', 'ig_revenue')

,quarter,quarter_label,ig_maus,ig_revenue,tk_maus,tk_revenue,ig_maus_z,ig_revenue_z,score
0,2024-03-31,Q2 2024,2221735781,12735807893,1871336709,4527462823,-0.989379,-0.989379,-0.890441
1,2024-06-30,Q3 2024,2242448858,12929389484,1631989456,3749413402,-0.622262,-0.622262,-0.560036
2,2024-09-30,Q4 2024,2276922459,13251575055,1576689951,3569650540,-0.011255,-0.011255,-0.010129
3,2024-12-31,Q1 2025,2369122780,14113266663,1495697320,3306366674,1.622896,1.622896,1.460607


In [13]:
# Formatar os números com separadores de milhar
lp_ig_maus = f"{ig_last_results['ig_maus'].values[0]:,}"
lp_tk_maus = f"{tk_last_results['tk_maus'].values[0]:,}"
lp_difference = f"{ig_last_results['ig_maus'].values[0] - tk_last_results['tk_maus'].values[0]:,}"

print('Instagram MAUs forecast:', lp_ig_maus)
print('TikTok MAUs forecast:', lp_tk_maus)
print('Difference between Instagram and TikTok:', lp_difference)
print()
if ig_last_results['ig_maus'].values[0] > tk_last_results['tk_maus'].values[0]:
    print('Instagram pretended to have more MAUs.')
else:
    print('TikTok pretended to have more MAUs.')

Instagram MAUs forecast: 2,369,122,780
TikTok MAUs forecast: 1,495,697,320
Difference between Instagram and TikTok: 873,425,460

Instagram pretended to have more MAUs.


In [14]:
# winner score
lp_ig_score = ig_last_results['ig_revenue'].values[0] / ig_last_results['ig_maus'].values[0]
lp_tk_score = tk_last_results['tk_revenue'].values[0] / tk_last_results['tk_maus'].values[0]

print('Instagram score:', lp_ig_score)
print('TikTok score:', lp_tk_score)
print()
if lp_ig_score > lp_tk_score:
    print('Instagram pretended to have a better score.')
else:
    print('TikTok pretended to have a better score.')

Instagram score: 5.957169793876195
TikTok score: 2.210585410422478

Instagram pretended to have a better score.


## Scoring: All Periods

In [15]:
display(forecast.drop(columns=['tk_maus', 'tk_revenue']))
display(forecast.drop(columns=['ig_maus', 'ig_revenue']))

,quarter,quarter_label,ig_maus,ig_revenue,ig_maus_z,ig_revenue_z,score
0,2024-03-31,Q2 2024,2221735781,12735807893,-0.989379,-0.989379,-0.890441
1,2024-06-30,Q3 2024,2242448858,12929389484,-0.622262,-0.622262,-0.560036
2,2024-09-30,Q4 2024,2276922459,13251575055,-0.011255,-0.011255,-0.010129
3,2024-12-31,Q1 2025,2369122780,14113266663,1.622896,1.622896,1.460607


,quarter,quarter_label,tk_maus,tk_revenue,ig_maus_z,ig_revenue_z,score
0,2024-03-31,Q2 2024,1871336709,4527462823,-0.989379,-0.989379,-0.890441
1,2024-06-30,Q3 2024,1631989456,3749413402,-0.622262,-0.622262,-0.560036
2,2024-09-30,Q4 2024,1576689951,3569650540,-0.011255,-0.011255,-0.010129
3,2024-12-31,Q1 2025,1495697320,3306366674,1.622896,1.622896,1.460607


In [16]:
ig_revenue = f"{forecast['ig_revenue'].sum():,}"
tk_revenue = f"{forecast['tk_revenue'].sum():,}"
difference = f"{forecast['ig_revenue'].sum() - forecast['tk_revenue'].sum():,}"

print('Instagram revenues forecast:', ig_revenue)
print('TikTok revenues forecast:', tk_revenue)
print('Difference between Instagram and TikTok:', difference)
print()
if forecast['ig_revenue'].sum() > forecast['tk_revenue'].sum():
    print('Instagram pretended to have more revenues.')
else:
    print('TikTok pretended to have more revenues.')

Instagram revenues forecast: 53,030,039,095
TikTok revenues forecast: 15,152,893,439
Difference between Instagram and TikTok: 37,877,145,656

Instagram pretended to have more revenues.


In [17]:
ig_maus = f"{forecast['ig_maus'].sum():,}"
tk_maus = f"{forecast['tk_maus'].sum():,}"
difference = f"{forecast['ig_maus'].sum() - forecast['tk_maus'].sum():,}"

print('Instagram MAUs forecast:', ig_maus)
print('TikTok MAUs forecast:', tk_maus)
print('Difference between Instagram and TikTok:', difference)
print()
if forecast['ig_maus'].sum() > forecast['tk_maus'].sum():
    print('Instagram pretended to have more MAUs.')
else:
    print('TikTok pretended to have more MAUs.')

Instagram MAUs forecast: 9,110,229,878
TikTok MAUs forecast: 6,575,713,436
Difference between Instagram and TikTok: 2,534,516,442

Instagram pretended to have more MAUs.


In [18]:
ig_score = forecast['ig_revenue'].sum() / forecast['ig_maus'].sum()
tk_score = forecast['tk_revenue'].sum() / forecast['tk_maus'].sum()

print('Instagram score:', ig_score)
print('TikTok score:', tk_score)
print()
if ig_score > tk_score:
    print('Instagram pretended to have a better score.')
else:
    print('TikTok pretended to have a better score.')

Instagram score: 5.82093314934462
TikTok score: 2.304372534855699

Instagram pretended to have a better score.
